In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np

# Experimental data

*Roitman, JD and Shadlen, MN (2002), “Response of neurons in the lateral intraparietal area during a combined visual discrimination reaction time task”, Journal of Neuroscience, Vol. 22(21), 9475-9489.*


#### additional information:
- 'coh'      coherence of trial (multiplied by 10 - ie. 32 is a coherence of 3.2%)
- 'correct'  whether the subject was correct (1 - correct, 0 - error)
- rt should be in ms (between 5 and 1762, but most in [200,1200])
- in Shinn et al. animal N (coded as 0 below) is used for all main figures
- per monkey and coherence level: trial number n>500 for animal 0, n>400 for animal 1.



In [72]:
ddm_data = pd.read_csv("../data/roitman_data_clean.csv")

In [73]:
ddm_data

,rt,coherence,decision,animal
0,464.0,256.0,1.0,0.0
1,318.0,64.0,1.0,0.0
2,531.0,128.0,1.0,0.0
3,567.0,0.0,1.0,0.0
4,398.0,0.0,1.0,0.0
...,...,...,...,...
6143,743.0,64.0,1.0,1.0
6144,704.0,0.0,1.0,1.0
6145,490.0,512.0,1.0,1.0
6146,558.0,256.0,1.0,1.0


# Generative model 
Use the *pyddm* toolbox:

https://pyddm.readthedocs.io/en/stable/

In [74]:
# load pyddm 

from pyddm import Model
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter
from pyddm.functions import fit_adjust_model, display_model

import pyddm.plot

from pyddm import Fittable
from pyddm.models import LossRobustBIC
from pyddm.functions import fit_adjust_model, fit_model

from pyddm.models import (
    BoundCollapsingExponential,
    DriftLinear,
    ICPointSourceCenter,
    NoiseConstant,
    OverlayNonDecision,
)


from roitman_utils import filter_roitman_data 
# this filters the data and puts it into the format we need for pyddm

In [75]:
data = filter_roitman_data(ddm_data, 
                            coherence=128, 
                            animal=0 , 
                            n_trial="all", 
                            attach_model_mask=False,
                            partition=None,
                            data_mode='pyddm')

In [76]:
model_fit = Model(name='Simple model (fitted)',
                        drift= DriftLinear(drift=Fittable(minval=0, maxval=5),t=0, x=Fittable(minval=-20, maxval=10)),
                        noise=NoiseConstant(noise=1),
                        bound=BoundCollapsingExponential(B=Fittable(minval=0.5, maxval=4), tau=Fittable(minval=0.1, maxval=4)),
                        overlay=OverlayNonDecision(nondectime=Fittable(minval=0.1, maxval=0.4)),
                        IC = ICPointSourceCenter(),
                        dx=.001, dt=.01, T_dur=2)

# fit model
fit_adjust_model(data, model_fit,
                fitting_method="differential_evolution",
                lossfunction=LossRobustBIC, verbose=False)

sol = model_fit.solve()


Info: Params [  2.47631418 -19.59667808   0.62346434   0.86244092   0.16327864] gave 108.81265713850348


In [77]:
# generate data 
generated_data = sol.resample(k=1000)

In [78]:
generated_data_corr = torch.tensor(generated_data.choice_upper, dtype=torch.float32)
generated_data_err = torch.tensor(generated_data.choice_lower, dtype=torch.float32)
generated_data = torch.cat([generated_data_corr, generated_data_err]).unsqueeze(-1)
real_data_corr = torch.tensor(data.choice_upper, dtype=torch.float32)
real_data_err = torch.tensor(data.choice_lower, dtype=torch.float32)
real_data = torch.cat([real_data_corr, real_data_err]).unsqueeze(-1)
gaussian_data = torch.tensor(np.random.normal(loc=data.choice_upper.mean(), scale=data.choice_upper.std(), size=len(generated_data)), dtype=torch.float32).unsqueeze(-1)

In [80]:
torch.save(generated_data, "../data/ddm/generated_data.pt")
torch.save(real_data, "../data/ddm/real_data.pt")
torch.save(gaussian_data, "../data/ddm/gaussian_data.pt")